In [1]:
'''
This notebook contains code to do knn imputation (with a tuned neighbour value using random search) on the pre_processed
data.

THIS TOOK A VERY LONG TIME TO RUN AND RESULTS WEREN'T GREAT SO NOT WORTH RUNNING AGAIN

'''

"\nThis notebook contains code to do knn imputation (with a tuned neighbour value using random search) on the pre_processed\ndata.\n\nTHIS TOOK A VERY LONG TIME TO RUN AND RESULTS WEREN'T GREAT SO NOT WORTH RUNNING AGAIN\n\n"

In [2]:
from utils import *
from models import *

from sklearn.neighbors import KNeighborsRegressor


In [5]:
train_data = load_app_training_data()
test_data = load_test_data()

Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


In [6]:
# pre-processing (No imputation or normalisation is applied)
train_data, test_data = encode_binary_cols(train_data, test_data)
train_data, test_data = one_hot_encode(train_data, test_data)
train_data, test_data = align_data(train_data, test_data)
train_Y = get_train_labels(train_data)
train_data, test_data = remove_days_employed_anomaly(train_data, test_data)
train_data, test_data = remove_missing_cols(train_data, test_data)

Label encoded columns ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
AFTER ONE HOT ENCODING
Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)
AFTER ALIGNMENT:
Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)
Test data contain 9274 anomalies out of 48744 rows
Removing columns with 0.68 proportion of missing values
AFTER REMOVING MISSING COLS (and aligning):
Training Features shape:  (307511, 232)
Testing Features shape:  (48744, 240)


In [7]:
def get_top_correlations(app_train, feature, n=15):
    '''
    Return a dataframe containing the top n features ranked by their pearson's correlations 
    (sorted by their absolute pcc score) 
    '''
    correlations = app_train.corr()[feature]
    correlations = abs(correlations).sort_values().tail(
        n)  # sort by correlation value (regardless if it's positive or negative)
    correlations = correlations.dropna()  # drop nans
    if 'TARGET' in correlations:
        correlations = correlations.drop(labels=[feature, 'TARGET'])  # remove corr to itself and target
    return correlations

In [8]:
def save_all_correlations(app_train):
    """
    For each feature will save a dataframe storing all correlations with the feature and every other feature
    """
    path = '../models/correlations/'
    for feature in train_data.columns[:]:
        correlations = app_train.corr()[feature]
        if '/' in feature or ':' in feature:
            # replace / and : with '' -> as they are invalid characters for filename 
            feature = feature.replace('/','')
            feature = feature.replace(':','')
        save_pickle(path+feature+'.pickle', correlations)
    print("ALL CORRELATIONS SAVED")

def top_correlations_from_all(feature:str, n):
    """
    Loads a previously saved dataframe containing feature corrleations and returns the top n correlated features
    (excludes target)
    """
    path = '../models/correlations/'
    filename = feature
    if '/' in filename or ':' in filename:
            # replace / and : with '' -> as they are invalid characters for filename 
            filename = filename.replace('/','')
            filename = filename.replace(':','')
            
    correlations = load_pickle(path+feature+'.pickle')
    correlations = abs(correlations).sort_values().tail(n)  # sort by correlation value (regardless if it's positive or negative)
    correlations = correlations.dropna()  # drop nans
    if 'TARGET' in correlations:
        correlations = correlations.drop(labels=[feature, 'TARGET'])  # remove corr to itself and target
    return correlations
    
# save_all_correlations(train_data)
# train_data.columns[181:][0].replace('/', '')


In [9]:
def top_corr_knn_imputation(feature, train, test, train_filled, test_filled,n_jobs):
    """
    Applies knn imputation for a single column. Will use random search to try find the best neighbour
    value (range 10-200). Returns a dataframes for the train and test with imputed column
    """
    top_corrs = top_correlations_from_all(feature, 15).keys()
    
    # create training data from columns with values
    not_null_mask = train[feature].notna()  # true if not nan = mask to get the values to train on
    knn_train_x = train_filled[not_null_mask][top_corrs] # get all top corr cols (minus target) which have a feature value
    knn_train_y = train_filled[not_null_mask][[feature]]
    
    # get the rows which require to be imputed for the features
    knn_test_x = train_filled[train[feature].isna()][top_corrs]  # only null values
    actual_test_x = test_filled[test[feature].isna()][top_corrs]  # only null values
    
    # train and predict
    filename = feature.replace(':', '')
    fileaname = feature.replace('/', '')
    try:
        neigh = load_pickle("../models/knn_corrs_rs_clf/"+filename+".pickle")
    except FileNotFoundError:
        print("Training col:", feature)
    #     neigh = KNeighborsRegressor(n_neighbors=75)
        model = KNeighborsRegressor()
        n_neighbors = range(10,200)
        hyperparameters = dict(n_neighbors=n_neighbors)
        neigh = RandomizedSearchCV(model, hyperparameters, n_iter=7, cv=3, verbose=3, n_jobs=n_jobs)
        neigh.fit(knn_train_x, knn_train_y)
        save_pickle("../models/knn_corrs_rs_clf/"+filename+".pickle", neigh)
        
    print('Best n_neigbors:', neigh.best_estimator_.get_params()['n_neighbors'])
    knn_test_y = neigh.predict(knn_test_x)
    # get the column to update nulls
    train_imputed_col = train[[feature]]
    train_imputed_col[train_imputed_col[feature].isna()] = knn_test_y
    
    # the train data has more unfilled cols than test, so sometimes imputation is not required
    try:
        actual_test_y = neigh.predict(actual_test_x)
        # get the column to update nulls for app_test
        test_imputed_col = test[[feature]]
        test_imputed_col[test_imputed_col[feature].isna()] = actual_test_y
    except ValueError:
        test_imputed_col = test[[feature]]
    
    return train_imputed_col, test_imputed_col


In [10]:
def impute_knn(app_train, app_test, n_jobs):
    """
    Applies knn imputation for all columns with missing values. KNN requires all other columns to be not null (so a 
    mean imputation is used to fill those columns in)
    """
    count = 1
    train_nan_cols = app_train.loc[:,app_train.isnull().any()]
    test_nan_cols = app_test.loc[:,app_test.isnull().any()]
    
    train_full_impute = app_train.copy()
    train_full_impute = train_full_impute.fillna(train_full_impute.mean())
    test_full_impute = app_test.copy()
    test_full_impute  = test_full_impute.fillna(train_full_impute.mean())

    print(list(train_nan_cols))
    for col in train_nan_cols: 
        imputed_train, imputed_test = top_corr_knn_imputation(col, app_train, app_test, train_full_impute, test_full_impute, n_jobs=n_jobs)
        app_train[col] = imputed_train
        app_test[col] = imputed_test
        print("{} out of {} done - col: {}".format(count, test_nan_cols.shape[1], col))
        count += 1
            
    return app_train, app_test


In [11]:
def log_reg_knn_imputation( app_train, app_test, model_save_path, predications_save_path, n_jobs):
    """
    Impute on all columns via knn then create predictions using log reg without any tuning on C
    """
    train, test = impute_knn(app_train, app_test, n_jobs=n_jobs)
#     save_pickle("../models/knn_impute_train_df1.pickle", train)
#     save_pickle("../models/knn_impute_test_df1.pickle", test)
    train = load_pickle("../models/knn_impute_train_df.pickle")
    test = load_pickle("../models/knn_impute_test_df.pickle")
    train_Y = train['TARGET']

    train = train.drop(columns=train.loc[:,train.isnull().any()].columns)  #
    trian, test, train_Y = align_data(train,test)
    # Drop the target and sk_id_curr from the training data
    if 'TARGET' in train:
        train = train.drop(columns=['TARGET'])
    train = train.drop(columns=['SK_ID_CURR'])  #
    test = test.drop(columns=['SK_ID_CURR'])  #
    
    model, preds = baseline_log_reg(train, train_Y, test, save_path=model_save_path)
    create_and_save_submission(test_data, preds, save_path=predications_save_path)

    
log_reg_knn_imputation(train_data.copy(), test_data.copy(),
                       model_save_path="../models/knn_imputation_top_corr_log_reg1.pickle",
                       predications_save_path='../test_predictions/knn_imputation_top_corr_log_reg1.csv',
                       n_jobs=10) 


['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DA

FileNotFoundError: [Errno 2] No such file or directory: '../models/correlations/AMT_ANNUITY.pickle'